Notebook to use the helper_CI tools to build the QED-CIS matrix:

\begin{equation}
\begin{bmatrix}
E_{ref} & 0 & 0 & \hbar {\bf g} \\
0 & {\bf A} +\Delta  & \hbar {\bf g}^{\dagger}  & \hbar {\bf G} \\
0 & \hbar {\bf g} & E_{ref} + \hbar \omega & 0 \\
\hbar {\bf g}^{\dagger} & \hbar {\bf G} & 0 & {\bf A} + \Delta + \hbar \Omega 
\end{bmatrix}
\begin{bmatrix}
{\bf c}^0_0 \\
{\bf c}^0_{ia} \\
{\bf c}^1_0 \\
{\bf c}^1_{ia}
\end{bmatrix}
=
E_{CISS-PF}
\begin{bmatrix}
{\bf c}^0_0 \\
{\bf c}^0_{ia} \\
{\bf c}^1_0 \\
{\bf c}^1_{ia},
\end{bmatrix}
\end{equation}

Here 

$$ E_{ref} = \langle R,0 | \hat{H}_e | R,0 \rangle  + \langle R,0 | \hat{H}_{dse} | R,0 \rangle$$

where $|R,0\rangle$ denotes the reference ket,

$$\langle R,0 | \hat{H}_{dse} | R,0 \rangle  =   \frac{1}{2} \langle R,0| ({\bf{\lambda}}\cdot  {\bf{\hat{\mu}}}_{\rm e} ) ^2 | R,0 \rangle \nonumber \\
    -({\bf{\lambda}}\cdot {\langle \bf{\hat{\mu}}}_{\rm e} \rangle ) \langle R,0|( {\bf{\lambda}}\cdot {\bf{\hat{\mu}}}_{\rm e} )| R,0 \rangle \\
    + \frac{1}{2} ( {\bf{\lambda}}\cdot \langle {\bf{\hat{\mu}}}_{\rm e}\rangle ) ^2 $$
    
and 

$$ \langle R,0 | \hat{H}_e | R,0 \rangle = \langle R,0 | \hat{V}_{ee} | R,0 \rangle + \langle R,0 | \hat{V}_{en} | R,0 \rangle
+ \langle R,0 | \hat{T}_{e} | R,0 \rangle + V_{nn} $$
    
**$E_{ref}$ should match the CQED-RHF Energy!**
   
Additionally, 
    
$$ {\bf A} = \langle S,n | \hat{H}_e | R,0 \rangle = \langle S,n | \hat{V}_{ee} | S,n \rangle + \langle S,n | \hat{V}_{en} | S,n \rangle
+ \langle S,n | \hat{T}_{e} | S,n \rangle + V_{nn}\mathbb{1} $$

$$ {\bf \Delta} =  \frac{1}{2} \langle S,n| ({\bf{\lambda}}\cdot  {\bf{\hat{\mu}}}_{\rm e} ) ^2 | S,n \rangle \nonumber \\
    -({\bf{\lambda}}\cdot {\langle \bf{\hat{\mu}}}_{\rm e} \rangle ) \langle S,n|( {\bf{\lambda}}\cdot {\bf{\hat{\mu}}}_{\rm e} )| S,n\rangle \\
    + \frac{1}{2} ( {\bf{\lambda}}\cdot \langle {\bf{\hat{\mu}}}_{\rm e}\rangle ) ^2 \mathbb{1} $$

$$ \hbar {\bf \Omega} =  \langle S,1 | \hat{H}_{p} | S,1 \rangle = \hbar \omega \langle S | S \rangle = \hbar \omega \mathbb{1} $$

where $|S,n\rangle$ denotes singly-excited electronic kets paired with photon state $|n\rangle$ where $n = 0,1$.

$$ {\bf g} = -\sqrt{\frac{\omega}{2}} \langle R,n| \hat{d}_e - \langle \hat{d} \rangle_e | S,m \rangle 
= -\sqrt{\frac{\omega}{2}}\langle R,n| \hat{d}_e| S,m \rangle $$
where $m \neq n$.

$$ {\bf G} = -\sqrt{\frac{\omega}{2}} \langle S,n| \hat{d}_e - \langle \hat{d} \rangle_e | S,m \rangle 
= -\sqrt{\frac{\omega}{2}}\langle S,n| \hat{d}_e| S,m \rangle + \sqrt{\frac{\omega}{2}} \langle \hat{d} \rangle _e\mathbb{1}  $$

A few things to keep in mind:

1. We need to offset the ${\bf G}$ block by the $\sqrt{\frac{\omega}{2}} \langle d \rangle_e \mathbb{1}$
2. We need to offset $E_{ref}$ and $E_{ref} + \hbar \omega$ by $E_{nuc} + \frac{1}{2} ( {\bf{\lambda}}\cdot \langle {\bf{\hat{\mu}}}_{\rm e}\rangle ) ^2$ 
3. We need to offset the ${\bf A}$ blocks by $E_{nuc} \mathbb{1}$
4. We need to offset the ${\Delta}$ blocks by $\frac{1}{2} ( {\bf{\lambda}}\cdot \langle {\bf{\hat{\mu}}}_{\rm e}\rangle ) ^2\mathbb{1}$
5. The CISS-Prism code defines the excitation energy relative to the CQED-RHF reference energy.  When we diagonlize the Hamiltonian as defined above, we will get the absolute energies of each CISS state.  For example, the lowest energy eigenstate $E_0$ will not typically be equal to $E_{ref}$.  To match the first excitation by diagonalizing the matrix above to the CISS-Prism result, we should do the following:  $E_1 - E_{ref}$

In [1]:
import time
import numpy as np
np.set_printoptions(precision=5, linewidth=200, suppress=True)
import psi4
from helper_cqed_rhf import cqed_rhf
# Compute size of Hamiltonian in GB
from scipy.special import comb
# prepare the determinant list
from helper_CI import Determinant, HamiltonianGenerator, compute_excitation_level
from itertools import combinations
from helper_cs_cqed_cis import *

In [2]:
def spin_idx_to_spat_idx_and_spin(P):
    """ function to take the numeric label of a spin orbital
        and return the spatial index and the spin index separately.
        Starts counting from 0:
        
        Arguments
        ---------
        P : int
            spin orbital label
        
        Returns
        -------
        [p, spin] : numpy array of ints
            p is the spatial orbital index and spin is the spin index.
            spin = 1  -> alpha
            spin = -1 -> beta
            
        Example
        -------
        >>> spin_idx_to_spat_idx_and_spin(0)
        >>> [0, 1]
        >>> spin_idx_to_spat_idx_and_spin(3)
        >>> [1, -1]
        
    """
    spin = 1
    if P % 2 == 0:
        p = P / 2
        spin = 1
    else:
        p = (P-1) / 2
        spin = -1
    return np.array([p, spin], dtype=int)


def map_spatial_to_spin(tei_spatial, I, J, K, L):
    """ function to take two electron integrals in the spatial orbital basis
        in chemist notation along with 4 indices I, J, K, L and return
        the corresponding two electron integral in the spin orbital basis
        in phycisit notation, <IJ||KL>
    
    """
    # Phys to Chem: <IJ||KL> -> [IK|JL] - [IL|JK]
    i_s = spin_idx_to_spat_idx_and_spin(I)
    k_s = spin_idx_to_spat_idx_and_spin(K)
    j_s = spin_idx_to_spat_idx_and_spin(J)
    l_s = spin_idx_to_spat_idx_and_spin(L)
    
    #print(i_s[1])
    # (ik|jl)
    spat_ikjl = tei_spatial[i_s[0], k_s[0], j_s[0], l_s[0]] * ( i_s[1] == k_s[1] ) *  ( j_s[1] == l_s[1] )
    
    # (il|jk)
    spat_iljk = tei_spatial[i_s[0], l_s[0], j_s[0], k_s[0]] * ( i_s[1] == l_s[1] ) *  ( j_s[1] == k_s[1] )
    
    return spat_ikjl - spat_iljk

def map_spatial_dipole_to_spin(mu, I, J, K, L):
    """ function to take two electron integrals in the spatial orbital basis
        in chemist notation along with 4 indices I, J, K, L and return
        the corresponding two electron integral in the spin orbital basis
        in phycisit notation, <IJ||KL>
    
    """
    # Phys to Chem: <IJ||KL> -> [IK|JL] - [IL|JK]
    i_s = spin_idx_to_spat_idx_and_spin(I)
    k_s = spin_idx_to_spat_idx_and_spin(K)
    j_s = spin_idx_to_spat_idx_and_spin(J)
    l_s = spin_idx_to_spat_idx_and_spin(L)
    
    #print(i_s[1])
    # (ik|jl)
    spat_ikjl = mu[i_s[0], k_s[0]] * mu[j_s[0], l_s[0]] * ( i_s[1] == k_s[1] ) *  ( j_s[1] == l_s[1] )
    
    # (il|jk)
    spat_iljk = mu[i_s[0], l_s[0]] * mu[j_s[0], k_s[0]] * ( i_s[1] == l_s[1] ) *  ( j_s[1] == k_s[1] )
    
    return spat_ikjl - spat_iljk

In [3]:

# Check energy against psi4?
compare_psi4 = True

# Memory for Psi4 in GB
# psi4.core.set_memory(int(2e9), False)
psi4.core.set_output_file('output.dat', False)

# Memory for numpy in GB
numpy_memory = 2

mol_str = """
Li 
H 1 1.0
symmetry c1
"""

options_dict = {'basis': 'sto-3g',
                  'scf_type': 'pk',
                  'e_convergence': 1e-10,
                  'd_convergence': 1e-10
                  }



# photon energy
omega_val = 4.75 / psi4.constants.Hartree_energy_in_eV

# define the lambda vector
lambda_vector = np.array([0.1, 0.1, 0.1])

mol = psi4.geometry(mol_str)


psi4.set_options(options_dict)

print('\nStarting SCF and integral build...')
t = time.time()

# First compute SCF energy using Psi4
scf_e, wfn = psi4.energy('SCF', return_wfn=True)

# now compute cqed-rhf to get transformation vectors with cavity
cqed_rhf_dict = cqed_rhf(lambda_vector, mol_str, options_dict)
# run cs_cqed_cis() for the three cases
cqed_cis_1 = cs_cqed_cis(lambda_vector, omega_val, mol_str, options_dict)



Starting SCF and integral build...

Start SCF iterations:

Canonical RHF One-electron energy = -13.3346745238277329
CQED-RHF One-electron energy      = -13.2560148284416996
Nuclear repulsion energy          = 1.5875316320100001
Dipole energy                     = 0.0402011708439930
SCF Iteration   1: Energy = -7.7171325856531112   dE = -7.71713E+00   dRMS = 6.48590E-03
SCF Iteration   2: Energy = -7.7191111990976129   dE = -1.97861E-03   dRMS = 9.50906E-04
SCF Iteration   3: Energy = -7.7191770057007894   dE = -6.58066E-05   dRMS = 3.01101E-04
SCF Iteration   4: Energy = -7.7191852843358841   dE = -8.27864E-06   dRMS = 1.27611E-04
SCF Iteration   5: Energy = -7.7191868740477094   dE = -1.58971E-06   dRMS = 5.90950E-05
SCF Iteration   6: Energy = -7.7191872323668242   dE = -3.58319E-07   dRMS = 2.88516E-05
SCF Iteration   7: Energy = -7.7191873213111215   dE = -8.89443E-08   dRMS = 1.45628E-05
SCF Iteration   8: Energy = -7.7191873446435100   dE = -2.33324E-08   dRMS = 7.49270E-06
SCF 

In [4]:
C = cqed_cis_1["CQED-CIS L VECTORS"]


C_g = C[:,0]
_n_ss = len(C_g)

pvac_idx = np.arange(0,_n_ss,2)
pone_idx = np.arange(1,_n_ss,2)

print(pvac_idx)
print(pone_idx)

_n_s = int(_n_ss/2)

C_g0 = C_g[pvac_idx]
C_g1 = C_g[pone_idx]

print(len(C_g0))
print(len(C_g1))
_c_n0r = np.reshape(C_g0, (_n_s,1))
_c_n1r = np.reshape(C_g1, (_n_s,1))

# get density matrix for each block
_D0 = np.outer(np.conj(_c_n0r.T), _c_n0r)
_D1 = np.outer(np.conj(_c_n1r.T), _c_n1r)

print("zero-photon D")
n_0 = np.trace(_D0)
n_1 = np.trace(_D1)
print(n_0)
print("one-photon D")
print(n_1)
print("total")
print(n_0 + n_1)

_nmo = wfn.nmo()
_ndocc = wfn.doccpi()[0]
_D1_ref = np.zeros((_nmo,_nmo))
for i in range(_ndocc):
    _D1_ref[i,i] = _D0[0,0] + _D1[0,0]
    
print(_D1_ref)

[ 0  2  4  6  8 10 12 14 16]
[ 1  3  5  7  9 11 13 15 17]
9
9
zero-photon D
(0.9883700091604682+0j)
one-photon D
(0.011629990839531737+0j)
total
(1+0j)
[[0.98766 0.      0.      0.      0.      0.     ]
 [0.      0.98766 0.      0.      0.      0.     ]
 [0.      0.      0.      0.      0.      0.     ]
 [0.      0.      0.      0.      0.      0.     ]
 [0.      0.      0.      0.      0.      0.     ]
 [0.      0.      0.      0.      0.      0.     ]]


/var/folders/5f/ftf5xlgj4dv_6xxqh_p9pwyc0000gq/T/ipykernel_7156/4066399987.py:40: ComplexWarning: Casting complex values to real discards the imaginary part
  _D1_ref[i,i] = _D0[0,0] + _D1[0,0]


In [5]:
excitations = []
_nvirt = _nmo - _ndocc
for i in range(_ndocc):
    for a in range(0, _nvirt):
        A = a + _ndocc
        excitations.append((i, A))
print(excitations)


_D1_RS = np.zeros((_nmo,_nmo))
_D1_SR = np.zeros((_nmo,_nmo))
for i in range(len(excitations)):
    ket_I = i+1
    _p = excitations[i][0]
    _q = excitations[i][1]
    _D1_RS[_p, _q] = _D0[0, ket_I] + _D1[0, ket_I]
    _D1_SR[_q, _p] = _D0[ket_I, 0] + _D1[ket_I, 0]
    
print(_D1_RS)
print(_D1_SR)

print(_D1_RS - _D1_SR.T)

[(0, 2), (0, 3), (0, 4), (0, 5), (1, 2), (1, 3), (1, 4), (1, 5)]
[[ 0.       0.      -0.00014  0.      -0.00009  0.00005]
 [ 0.       0.      -0.01474  0.       0.0223  -0.00518]
 [ 0.       0.       0.       0.       0.       0.     ]
 [ 0.       0.       0.       0.       0.       0.     ]
 [ 0.       0.       0.       0.       0.       0.     ]
 [ 0.       0.       0.       0.       0.       0.     ]]
[[ 0.       0.       0.       0.       0.       0.     ]
 [ 0.       0.       0.       0.       0.       0.     ]
 [-0.00014 -0.01474  0.       0.       0.       0.     ]
 [ 0.       0.       0.       0.       0.       0.     ]
 [-0.00009  0.0223   0.       0.       0.       0.     ]
 [ 0.00005 -0.00518  0.       0.       0.       0.     ]]
[[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]


/var/folders/5f/ftf5xlgj4dv_6xxqh_p9pwyc0000gq/T/ipykernel_7156/3042451891.py:16: ComplexWarning: Casting complex values to real discards the imaginary part
  _D1_RS[_p, _q] = _D0[0, ket_I] + _D1[0, ket_I]
/var/folders/5f/ftf5xlgj4dv_6xxqh_p9pwyc0000gq/T/ipykernel_7156/3042451891.py:17: ComplexWarning: Casting complex values to real discards the imaginary part
  _D1_SR[_q, _p] = _D0[ket_I, 0] + _D1[ket_I, 0]


In [6]:
_D1_SS = np.zeros((_nmo, _nmo))
for _I in range(len(excitations)):
    _ket_I = _I + 1
    _i = excitations[_I][0]
    _a = excitations[_I][1]

    for _J in range(len(excitations)):
        _ket_J = _J + 1
        _j = excitations[_J][0]
        _b = excitations[_J][1]
        _D1_SS[_a, _b] += (_D0[_ket_I, _ket_J] + _D1[_ket_I, _ket_J]) * (_i == _j)
        _D1_SS[_i, _j] += (_D0[_ket_I, _ket_J] + _D1[_ket_I, _ket_J]) * (_a == _b)
        


/var/folders/5f/ftf5xlgj4dv_6xxqh_p9pwyc0000gq/T/ipykernel_7156/4098309663.py:11: ComplexWarning: Casting complex values to real discards the imaginary part
  _D1_SS[_a, _b] += (_D0[_ket_I, _ket_J] + _D1[_ket_I, _ket_J]) * (_i == _j)
/var/folders/5f/ftf5xlgj4dv_6xxqh_p9pwyc0000gq/T/ipykernel_7156/4098309663.py:12: ComplexWarning: Casting complex values to real discards the imaginary part
  _D1_SS[_i, _j] += (_D0[_ket_I, _ket_J] + _D1[_ket_I, _ket_J]) * (_a == _b)


In [7]:
print(_D1_SS)

_D1 = _D1_ref + _D1_RS + _D1_SR + _D1_SS
print(_D1)

print(np.trace(_D1))

[[ 0.00002 -0.00036  0.       0.       0.       0.     ]
 [-0.00036  0.01232  0.       0.       0.       0.     ]
 [ 0.       0.       0.00112  0.       0.00276  0.00018]
 [ 0.       0.       0.       0.       0.       0.     ]
 [ 0.       0.       0.00276  0.       0.01118  0.00022]
 [ 0.       0.       0.00018  0.       0.00022  0.00004]]
[[ 0.98768 -0.00036 -0.00014  0.      -0.00009  0.00005]
 [-0.00036  0.99998 -0.01474  0.       0.0223  -0.00518]
 [-0.00014 -0.01474  0.00112  0.       0.00276  0.00018]
 [ 0.       0.       0.       0.       0.       0.     ]
 [-0.00009  0.0223   0.00276  0.       0.01118  0.00022]
 [ 0.00005 -0.00518  0.00018  0.       0.00022  0.00004]]
1.9999999999999996


Parse the `cqed_rhf_dict` dictionary for useful quantities.

In [ ]:
# grab necessary quantities from cqed_rhf_dict
rhf_reference_energy = cqed_rhf_dict["RHF ENERGY"]
cqed_reference_energy = cqed_rhf_dict["CQED-RHF ENERGY"]
C = cqed_rhf_dict["CQED-RHF C"]
D = cqed_rhf_dict["CQED-RHF DENSITY MATRIX"]
eps = cqed_rhf_dict["CQED-RHF EPS"]
dc = cqed_rhf_dict["DIPOLE ENERGY (1/2 (\lambda \cdot <\mu>_e)^2)"]

# collect rhf wfn object as dictionary
wfn_dict = psi4.core.Wavefunction.to_file(wfn)

# update wfn_dict with orbitals from CQED-RHF
wfn_dict["matrix"]["Ca"] = C
wfn_dict["matrix"]["Cb"] = C
# update wfn object
wfn = psi4.core.Wavefunction.from_file(wfn_dict)

# Grab data from wavfunction class
Ca = wfn.Ca()
ndocc = wfn.doccpi()[0]
nmo = wfn.nmo()

print(nmo)
print(ndocc)

mints = psi4.core.MintsHelper(wfn.basisset())


We will build $H_{core}$ in the AO basis.  

$$ {\bf H}_{core} = {\bf T} + {\bf V} + {\bf q}_{PF} + {\bf d}_{PF} $$ 

where ${\bf q}_{PF}$ has elements

$$ q_{\mu \nu} = -\frac{1}{2} \sum_{\xi, \xi'} \lambda^{\xi} \lambda^{\xi'} q_{\mu \nu}^{\xi \xi'} $$ 

and ${\bf d}_{PF}$ has elements

$$ \tilde{d}_{\mu \nu} = - {\bf \lambda} \cdot \langle \hat{\bf \mu}_e \rangle d_{\mu \nu} $$
with $d_{\mu \nu} = {\bf \lambda} \cdot \hat{\bf \mu}_e$. 

We can grab these arrays from the cqed-rhf dictionary as follows:

${\bf q}_{PF}$ : `cqed_rhf_dict["PF 1-E QUADRUPOLE MATRIX AO"]`

${\bf d}_{PF}$ : `cqed_rhf_dict["PF 1-E SCALED DIPOLE MATRIX AO"]`

${\bf T}$ : `cqed_rhf_dict["1-E KINETIC MATRIX AO"]`

${\bf V}$ : `cqed_rhf_dict["1-E POTENTIAL MATRIX AO"]`



In [ ]:
T = cqed_rhf_dict["1-E KINETIC MATRIX AO"]
V = cqed_rhf_dict["1-E POTENTIAL MATRIX AO"]
q_PF = cqed_rhf_dict["PF 1-E QUADRUPOLE MATRIX AO"]
d_PF = cqed_rhf_dict["PF 1-E SCALED DIPOLE MATRIX AO"]

# preparing 1e- array
H = T + V + q_PF + d_PF

# Update H, transform to MO basis and tile for alpha/beta spin
H = np.einsum('uj,vi,uv', Ca, Ca, H)

H = np.repeat(H, 2, axis=0)
print("printing H after 1st repeat")
print(H)
H = np.repeat(H, 2, axis=1)
print("printing H after 2nd repeat")
print(H)

spin_ind = np.arange(H.shape[0], dtype=int) % 2
H *= (spin_ind.reshape(-1, 1) == spin_ind)

print(H)

Now we will build the $\langle pq||rs\rangle$ integrals augmented
by 2-electron scaled dipole integrals in the spin orbital 
basis which are (nso x nso x nso x nso) = (2 * nao x 2 * nao x 2 * nao x 2 * nao).

First we will grab the ${\bf d} = \lambda \cdot \mu$ integrals in the CQED-RHF MO basis from the CQED-RHF dictionary as

${\bf d}_{MO}$ : `cqed_rhf_dict["PF 1-E DIPOLE MATRIX MO"]`

In [ ]:
l_dot_mu_el_cmo = cqed_rhf_dict["PF 1-E DIPOLE MATRIX MO"]

In [ ]:


print('\nTotal time taken for ERI integrals: %.3f seconds.\n' % (time.time() - t))

#Make spin-orbital MO
print('Starting AO -> spin-orbital MO transformation...')
t = time.time()
MO = np.asarray(mints.mo_spin_eri(Ca, Ca))

nso = 2 * nmo
TDI_spin = np.zeros((nso, nso, nso, nso))
for i in range(nso):
    for j in range(nso):
        for k in range(nso):
            for l in range(nso):
                TDI_spin[i, j, k, l] = map_spatial_dipole_to_spin(l_dot_mu_el_cmo, i, j, k, l)
                

MO += TDI_spin

print(F'The dimensions of the 1-e H matrix is {H.shape}')
print(F'The dimesions of the 2-e integral arrays in the spin orbital basis is {MO.shape}')

Now we are going to build ${\bf g}_{PF}$, defined as 


$${\bf g}_{PF} = -\sqrt{\frac{\omega}{2}} {\bf d}$$

In [ ]:
g_mat = -np.sqrt(omega_val / 2) * l_dot_mu_el_cmo


Now we are going to put the 1-e arrays into the spin-orbital basis, so that each in the resulting
arrays is of the form $H_{pq} = \langle \chi_p | \hat{O}_1 | \chi_q \rangle$.  

In [ ]:

g_mat = np.repeat(g_mat, 2, axis=0)
g_mat = np.repeat(g_mat, 2, axis=1)
print("printing g_mat after 2nd repeate")
print(g_mat)


# Make H block diagonal

g_mat *= (spin_ind.reshape(-1, 1) == spin_ind)

print("after spin blocking")
print(g_mat)

print('..finished transformation in %.3f seconds.\n' % (time.time() - t))

Here we are going to generate the set of all singly-excited determinants $|\Phi_i^a\rangle$
that we will call `singlesDets` and the reference determinant $|\Phi_0\rangle$ 
that we will calle `refDet`.

In [ ]:


print('Generating singly-excited determinants')
t = time.time()
singlesDets = []
for alpha in combinations(range(nmo), ndocc):
    alpha_ex_level = compute_excitation_level(alpha, ndocc)
    for beta in combinations(range(nmo), ndocc):
        beta_ex_level = compute_excitation_level(beta, ndocc)
        if alpha_ex_level + beta_ex_level == 1:
            print(F' adding alpha: {alpha} and beta: {beta}\n') 
            singlesDets.append(Determinant(alphaObtList=alpha, betaObtList=beta))

print('..finished generating singly-excited determinants in %.3f seconds.\n' % (time.time() - t))
print(F'..there are {len(singlesDets)} determinants \n')
for i in range(len(singlesDets)):
    print(singlesDets[i])
    
print('Generating reference determinant')

occList = [i for i in range(ndocc)]
refDet = Determinant(alphaObtList=occList, betaObtList=occList)
print(refDet)

# get the number of singles
n_s = len(singlesDets)

##### this illustrates some functionality of the determinant class.
numDet = len(detList)
print(numDet)
for i in range(numDet):
    for j in range(numDet):
        unique1, unique2, sign = detList[i].getUniqueOrbitalsInMixIndexListsPlusSign(detList[j])
        print(detList[i], detList[j], unique1, unique2, sign)
        


First we are going to generate the ${\bf A + \Delta} + n\hbar{\bf \Omega}$.
    
$$ {\bf A} = \langle S,n | \hat{H}_e | R,0 \rangle = \langle S,n | \hat{V}_{ee} | S,n \rangle + \langle S,n | \hat{V}_{en} | S,n \rangle
+ \langle S,n | \hat{T}_{e} | S,n \rangle + V_{nn}\mathbb{1} $$

$$ {\bf \Delta} =  \frac{1}{2} \langle S,n| ({\bf{\lambda}}\cdot  {\bf{\hat{\mu}}}_{\rm e} ) ^2 | S,n \rangle \nonumber \\
    -({\bf{\lambda}}\cdot {\langle \bf{\hat{\mu}}}_{\rm e} \rangle ) \langle S,n|( {\bf{\lambda}}\cdot {\bf{\hat{\mu}}}_{\rm e} )| S,n\rangle \\
    + \frac{1}{2} ( {\bf{\lambda}}\cdot \langle {\bf{\hat{\mu}}}_{\rm e}\rangle ) ^2 \mathbb{1} $$

In [ ]:

print('Generating A+\Delta sublock...')
A_D_generator = HamiltonianGenerator(H, MO)


A_D_matrix = A_D_generator.generateMatrix(singlesDets)
print("printing A+\Delta")
print(A_D_matrix)

print('Generating E_nuc I sublock...')
Enuc = cqed_rhf_dict["NUCLEAR REPULSION ENERGY"]
Enuc_matrix = np.identity(n_s) * Enuc
print(Enuc_matrix)

print('Generating \Omega I sublock...')
Omega_matrix = np.identity(n_s) * omega_val
print(Omega_matrix)


print('Generating dc I sublock...')
dc_matrix = np.identity(n_s) * dc
print(dc_matrix)

Next we are going to generate the ${\bf G}$ blocks that arise from
$$ {\bf G} = -\sqrt{\frac{\omega}{2}} \langle S,n| \hat{d}_e - \langle \hat{d} \rangle_e | S,m \rangle 
= -\sqrt{\frac{\omega}{2}}\langle S,n| \hat{d}_e| S,m \rangle + \sqrt{\frac{\omega}{2}} \langle \hat{d} \rangle _e\mathbb{1}  $$

In [ ]:
print('Generating G sublock...')
g_generator = HamiltonianGenerator(g_mat, MO * 0)

G_matrix = g_generator.generateMatrix(singlesDets)
print("printing G")
print(G_matrix)

# get <d>
d_exp = cqed_rhf_dict["EXPECTATION VALUE OF d"]
# form \sqrt{omega/2} * <d> I 
G_exp_matrix = np.sqrt(omega_val / 2) * np.identity(n_s) * d_exp
print("printing <G>I")
print(G_exp_matrix)

print("printing G+<G>I")
print(G_matrix + G_exp_matrix)

Next we are going to generate the ${\bf g}$ blocks which arise from 

$$ {\bf g} = -\sqrt{\frac{\omega}{2}} \langle R,n| \hat{d}_e - \langle \hat{d} \rangle_e | S,m \rangle 
= -\sqrt{\frac{\omega}{2}}\langle R,n| \hat{d}_e| S,m \rangle $$
where $m \neq n$.




In [ ]:
g_vec = np.zeros((1,n_s))
for i in range(n_s):
    g_vec[0,i] = g_generator.calcMatrixElement(refDet, singlesDets[i])
print(g_vec)
g_dag = np.conj(g_vec).T
print(g_dag)

Generate $E_0$

In [ ]:
E_electronic_ref = A_D_generator.calcMatrixElement(refDet, refDet)
print(E_electronic_ref)
E_0 = E_electronic_ref + Enuc + dc
print(E_0)
print(cqed_reference_energy)


Now build the entire matrix

In [ ]:
# define the offsets
R0_offset = 0
S0_offset = 1
R1_offset = n_s + 1
S1_offset = n_s + 2


H_CISS_PF = np.zeros((n_s * 2 + 2, n_s * 2 + 2))
H_label = np.zeros((n_s * 2 + 2, n_s * 2 + 2),dtype=str)

H_CISS_PF[R0_offset, R0_offset] = E_0
H_label[R0_offset, R0_offset] ='E'
H_CISS_PF[R0_offset:S0_offset, S1_offset:] = g_vec
H_label[R0_offset:S0_offset, S1_offset:] = 'g'


H_CISS_PF[S0_offset:R1_offset, R1_offset:S1_offset] = g_dag
H_label[S0_offset:R1_offset, R1_offset:S1_offset] = 'g'

H_CISS_PF[R1_offset:S1_offset, S0_offset:R1_offset] = g_vec 
H_label[R1_offset:S1_offset, S0_offset:R1_offset] = 'g'

H_CISS_PF[S1_offset:,          R0_offset:S0_offset] = g_dag
H_label[S1_offset:,          R0_offset:S0_offset] = 'g'

H_CISS_PF[S0_offset:R1_offset, S0_offset:R1_offset] = A_D_matrix + Enuc_matrix + dc_matrix
H_label[S0_offset:R1_offset, S0_offset:R1_offset] = 'a'

H_CISS_PF[R1_offset, R1_offset] = E_0 + omega_val
H_label[R1_offset, R1_offset] = 'o'


H_CISS_PF[S1_offset:, S1_offset:] = A_D_matrix + Enuc_matrix + dc_matrix + Omega_matrix
H_label[S1_offset:, S1_offset:] = 'A'

H_CISS_PF[S1_offset:,S0_offset:R1_offset] = G_matrix+G_exp_matrix
H_label[S1_offset:,S0_offset:R1_offset] = 'G'

H_CISS_PF[S0_offset:R1_offset, S1_offset:] = G_matrix+G_exp_matrix
H_label[S0_offset:R1_offset, S1_offset:] = 'G'

print("Printing the Hamiltonian schematic")
print(H_label)

print("Printing the Hamiltonian matrix")
print(H_CISS_PF)



In [ ]:
print('..finished generating Matrix in %.3f seconds.\n' % (time.time() - t))

print('Diagonalizing Hamiltonian Matrix...')

t = time.time()
#print(E_0)
e_fci, wavefunctions = np.linalg.eigh(H_CISS_PF)
print("|G> is ",e_fci[0])
print("|LP> is ",e_fci[1])

for i in range(len(e_fci)):
    #ex_e = (e_fci[i]-E_0)*27.211
    print(e_fci[i])
    #print(F'{ex_e:.12e}')
    

#expected_e0 = -6.607231755184198
#expected_e1 = -6.432685934463518


expected_LiH2p_eref = -6.607149667909853
expected_LiH2p_e0 = -6.607231755184202
expected_LiH2p_e1 = -6.432685934463522

#assert np.isclose(E_0, expected_LiH2p_eref)
#assert np.isclose(expected_LiH2p_e0, e_fci[0])
#assert np.isclose(expected_LiH2p_e1, e_fci[1])


expected_LiH_eref = -7.719187353322769
expected_LiH_e0 = -7.724724046298468
expected_LiH_e2 = -7.565785189439455

assert np.isclose(E_0, expected_LiH_eref)
assert np.isclose(expected_LiH_e0, e_fci[0])
assert np.isclose(expected_LiH_e2, e_fci[2])


